In [1]:
!git clone https://github.com/lens-protocol/api-examples.git

Cloning into 'api-examples'...


In [2]:
import os

In [4]:
root, dirs, files =next(os.walk('graphql'))

In [5]:
files

['add-reaction.graphql',
 'approve-module.graphql',
 'approved-allowance-of-modules.graphql',
 'authenticate.graphql',
 'broadcast.graphql',
 'burn-profile.graphql',
 'challenge.graphql',
 'collect.graphql',
 'comment.graphql',
 'common.graphql',
 'create-comment-via-dispatcher.graphql',
 'create-mirror-via-dispatcher.graphql',
 'create-post-via-dispatcher.graphql',
 'create-profile.graphql',
 'create-set-profile-image-uri-via-dispatcher.graphql',
 'create-set-profile-metadata-via-dispatcher.graphql',
 'dispatcher.graphql',
 'does-follow.graphql',
 'enabled-modules-currencies.graphql',
 'enabled-modules.graphql',
 'explore-profiles.graphql',
 'explore-publications.graphql',
 'follow.graphql',
 'follower-nft-owned-token-ids.graphql',
 'followers.graphql',
 'following.graphql',
 'generated.ts',
 'get-default-profile.graphql',
 'get-profile.graphql',
 'get-profiles.graphql',
 'get-publication.graphql',
 'get-publications.graphql',
 'get-users-nfts.graphql',
 'global-protocol-stats.graphql

In [7]:
sql_queries = []

In [8]:
for file in files:
    with open(os.path.join(root,file)) as f:
        sql_query = f.read()
        sql_queries.append(sql_query)

In [13]:
for i,sql_query in enumerate(sql_queries):
    
    if sql_query.count('$request') != 2:
        print(files[i])

common.graphql
enabled-modules-currencies.graphql
enabled-modules.graphql
generated.ts
ping.graphql
proxy-action-status.graphql
recommended-profiles.graphql


In [16]:
sql_queries_relevant = list(filter(lambda x: x.count('$request') ==2, sql_queries))
print(len(sql_queries_relevant))

61


In [70]:
sql_queries_edited = []
names = []
for sql_query in sql_queries_relevant:
    # print(sql_query)
    first_index = sql_query.find('$request')
    first_end_index = sql_query.find('!)',first_index)
    
    sql_query_edited = sql_query[0:first_index-1]+sql_query[first_end_index+2:]
    sql_query_edited = sql_query_edited.replace('{','{{')
    sql_query_edited = sql_query_edited.replace('}','}}')
    
    sql_query_edited = sql_query_edited.replace(
        '$request',
        """
        {{
        {var_dict}}}
        """)
    
    
    name = sql_query_edited.split()[1]
    if name != 'CreateSetProfileMetadataViaDispatcher(':
        
        names.append(name)
    
        sql_queries_edited.append(sql_query_edited)

In [71]:
import textwrap
wrapper = textwrap.TextWrapper(initial_indent = '\t', subsequent_indent = '\t')

In [72]:
with open('queries'+'.py','w') as f:
    for name, sql_query in zip(names, sql_queries_edited):
    
    
        f.write('def '+ name + '(var_dict)'+':'+'\n')
        
        f.write('\t'+'x=' + '"')
        f.write(sql_query.replace('\n','').replace('\t',''))
        f.write('"'+".format(var_dict=var_dict)"+'\n'+'\t')
        f.write('return x' + '\n'+'\n')